In [ ]:
import xarray as xr
import fsspec
import hvplot.xarray

In [ ]:
from dask_gateway import Gateway
from dask.distributed import Client
gateway = Gateway()
# see Gateway options to use in new_cluster by doing: gateway.cluster_options()
cluster = gateway.new_cluster(environment='pangeo', profile='Small Worker')  
cluster.scale(20)
client = Client(cluster)
cluster

In [ ]:
account_name = 'daymeteuwest'
container_name = 'daymet-zarr'
store = fsspec.get_mapper('az://' + container_name + '/daily/na.zarr', account_name=account_name)
# consolidated=True speeds of reading the metadata
ds = xr.open_zarr(store, consolidated=True)

In [ ]:
ds.tmax

In [ ]:
ds_sub = ds.sel(y=slice(-319992.78652461, -331137.92646756), x=slice(-877842.45533692, -864410.28467228))

In [ ]:
ds_sub.tmax.groupby('time.month').mean()

In [ ]:
%%time
dt = ds_sub.tmax.mean(dim=['x','y']).compute()

In [ ]:
dt.hvplot(x='time', grid=True)

In [ ]:
client.close(); cluster.shutdown()